In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/danielmiessler/SecLists/master/Passwords/Common-Credentials/10-million-password-list-top-1000000.txt"

df_in = pd.read_csv(url, header=None)

In [2]:
df_in = pd.read_csv(url, header=None)def add_space(x):
    return " ".join(x)

df = df_in[0].dropna().apply(lambda x: add_space(x) + "~~")# apply(lambda x: " ".join(letter for letter in x) + "~")

In [3]:
df

0               1 2 3 4 5 6~~
1           p a s s w o r d~~
2           1 2 3 4 5 6 7 8~~
3               q w e r t y~~
4         1 2 3 4 5 6 7 8 9~~
                 ...         
999993          v j h t 0 8~~
999994      V j h t 0 4 0 9~~
999995          v j h t 0 4~~
999996          v j h t 0 1~~
999997        v j h t 0 0 8~~
Name: 0, Length: 999996, dtype: object

In [4]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import tensorflow as tf 

# Create a Tokenizer object
tokenizer = Tokenizer()

# Fit the Tokenizer on the sentences
tokenizer.fit_on_texts(df)

# Convert the sentences to tokens
tokens = tokenizer.texts_to_sequences(df)

# Pad the tokens to the same length
padded_tokens = pad_sequences(tokens, padding="post")

# Generate a training dataset
train_dataset = tf.data.Dataset.from_tensor_slices(padded_tokens)

In [5]:
padded_tokens

array([[ 2,  8, 14, ...,  0,  0,  0],
       [31,  1, 10, ...,  0,  0,  0],
       [ 2,  8, 14, ...,  0,  0,  0],
       ...,
       [32, 35, 25, ...,  0,  0,  0],
       [32, 35, 25, ...,  0,  0,  0],
       [32, 35, 25, ...,  0,  0,  0]])

In [12]:
# Hyperparameters
d_model = 512
dff=2048
maximum_position_encoding = 10000

# Size of input vocab plus start and end tokens
input_vocab_size = tokenizer_pt.vocab_size + 2
target_vocab_size = tokenizer_en.vocab_size + 2

# Encoder ##################################
input = tf.keras.layers.Input(shape=(None,))
x = tf.keras.layers.Embedding(input_vocab_size, d_model)(input)

## positional encoding
scaling_factor = tf.keras.backend.constant(np.sqrt(d_model), shape = (1,1,1))
x = tf.keras.layers.Multiply()([x,scaling_factor])
pos = positional_encoding(maximum_position_encoding, d_model)
x = tf.keras.layers.Add()([x, pos[: , :tf.shape(x)[1], :]] )

## self-attention
query = tf.keras.layers.Dense(d_model)(x)
value = tf.keras.layers.Dense(d_model)(x)
key = tf.keras.layers.Dense(d_model)(x)
attention = tf.keras.layers.Attention()([query, value, key])
attention = tf.keras.layers.Dense(d_model)(attention)
x = tf.keras.layers.Add()([x , attention]) # residual connection
x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

## Feed Forward
dense = tf.keras.layers.Dense(dff, activation='relu')(x)
dense = tf.keras.layers.Dense(d_model)(dense)
x = tf.keras.layers.Add()([x , dense])     # residual connection
encoder = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)


# Decoder ##################################
target = tf.keras.layers.Input(shape=(None,))
x = tf.keras.layers.Embedding(target_vocab_size, d_model)(target)

## positional encoding
x = tf.keras.layers.Multiply()([x,scaling_factor])
pos = positional_encoding(maximum_position_encoding, d_model)
x = tf.keras.layers.Add()([x, pos[: , :tf.shape(x)[1], :] ])           

## self-attention
query = tf.keras.layers.Dense(d_model)(x)
value = tf.keras.layers.Dense(d_model)(x)
key = tf.keras.layers.Dense(d_model)(x)
attention = tf.keras.layers.Attention(causal = True)([query, value, key])
attention = tf.keras.layers.Dense(d_model)(attention)
x = tf.keras.layers.Add()([x , attention])  # residual connection
x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

## encoder-decoder attention
query = tf.keras.layers.Dense(d_model)(x)
value = tf.keras.layers.Dense(d_model)(encoder)
key = tf.keras.layers.Dense(d_model)(encoder)
attention = tf.keras.layers.Attention()([query, value, key])
attention = tf.keras.layers.Dense(d_model)(attention)
x = tf.keras.layers.Add()([x , attention])  # residual connection
x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

## Feed Forward
dense = tf.keras.layers.Dense(dff, activation='relu')(x)
dense = tf.keras.layers.Dense(d_model)(dense)
x = tf.keras.layers.Add()([x , dense])      # residual connection
decoder = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

######################################################

x = tf.keras.layers.Dense(target_vocab_size)(decoder)
base_model = tf.keras.models.Model(inputs=[input,target], outputs=x)
base_model.summary()

NameError: name 'tokenizer_pt' is not defined

In [ ]:
base_model.fit(x = [input, target[:, :-1]], y = target[:, 1:], epochs=20)

In [ ]:
result = [tokenizer_en.vocab_size]for _ in range(40):
   predict = model.predict([input ,np.asarray([result])])
   result.append(np.argmax(predict[-1,-1]))
   if result[-1] == tokenizer_en.vocab_size + 1:
       break

In [8]:
import tensorflow as tf

query = tf.keras.layers.Input(shape=(None,3,))
value = tf.keras.layers.Input(shape=(4,2,)) 
key = tf.keras.layers.Input(shape=(4,3,))

x = tf.keras.layers.Attention()([query, value, key])
model = tf.keras.models.Model(inputs=[query, value, key], outputs=x)

temp_k = tf.constant([[[10,0,0],
                       [0,10,0],
                       [0,0,10],
                       [0,0,10]]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[[   1,0],
                       [  10,0],
                       [ 100,5],
                       [1000,6]]], dtype=tf.float32)  # (4, 2)

temp_q = tf.constant([[[0, 10, 0]]], dtype=tf.float32)  # (1, 3)

preds = model.predict([temp_q,temp_v,temp_k])

1/1 [==============================] - 0s 48ms/step
